In [1]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import itertools
import scipy.stats as stats
from scipy.stats.mstats import winsorize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from xgboost import XGBClassifier, XGBRegressor

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [2]:
#Import 2015 and 2017 census data, change column names to be consistent across datasets

df2015 = pd.read_csv('acs2015_census_tract_data.csv')

df2017 = pd.read_csv('acs2017_census_tract_data.csv')
df2017.rename({'TractId': 'CensusTract', 'VotingAgeCitizen': 'Citizen'}, axis=1, inplace=True)

In [3]:
#Change count statistics to percentages based on total population; remove %women because of perfect multicollinearity with %men

df2015['Men'] = df2015['Men']/df2015['TotalPop']
df2015['Citizen'] = df2015['Citizen']/df2015['TotalPop']
df2015['Employed'] = df2015['Employed']/df2015['TotalPop']
df2015.drop(['CensusTract', 'Women'], axis=1, inplace=True)

df2017['Men'] = df2017['Men']/df2017['TotalPop']
df2017['Citizen'] = df2017['Citizen']/df2017['TotalPop']
df2017['Employed'] = df2017['Employed']/df2017['TotalPop']
df2017.drop(['CensusTract', 'Women'], axis=1, inplace=True)

In [4]:
#Create df_diff as change from 2015 to 2017
#Add Poverty_Change variable as categorical, -1=significant decrease in poverty, 0=no significant change, +1=significant increase in poverty

df_diff = df2017.copy()
df_diff.drop(['State', 'County'], axis=1, inplace=True)
for col in df_diff:
        df_diff[col] = df2017[col] - df2015[col]
df_diff['Poverty_Change'] = 0
df_diff.loc[(df_diff['Poverty'] >= 1.5), 'Poverty_Change'] = 1
df_diff.loc[(df_diff['Poverty'] <= -1.5), 'Poverty_Change'] = -1

#Add poverty change to 2015 dataset
df2015['Poverty_Change'] = df_diff['Poverty_Change']
df2015['Poverty_Change_val'] = df_diff['Poverty']

df2015['Poverty_Change_2c'] = 0
df2015.loc[(df2015['Poverty_Change_val'] < 0), 'Poverty_Change_2c'] = 1

In [5]:
#Create PCA components to capture 50% of the variation in 'State' and 'County' variables

pca = PCA(n_components=10)
principal_components = pca.fit_transform(pd.concat([pd.get_dummies(df2015['State']), pd.get_dummies(df2015['County'])], axis=1))

principalDf = pd.DataFrame(data = principal_components)
df2015 = pd.concat([df2015, principalDf], axis=1)

In [6]:
#Drop null values

df2015.dropna(inplace=True)
df_diff.dropna(inplace=True)

In [7]:
#Winsorize outliers

for col in df2015.columns[2:]:
    threshold = 5
    q75, q25 = np.percentile(df2015[col], [75,25])
    iqr = q75-q25
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)

    outliers = df2015[(df2015[col] > max_val) | (df2015[col] < min_val)]
    print(col)
    print('There are {} tracts identified as outliers.'.format(outliers.shape[0]))

    df2015[col] = pd.Series(winsorize(df2015[col], limits = (outliers.shape[0]/df2015[col].shape[0])))

for col in df_diff.columns:
    threshold = 3
    q75, q25 = np.percentile(df_diff[col], [75,25])
    iqr = q75-q25
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)

    outliers = df_diff[(df_diff[col] > max_val) | (df_diff[col] < min_val)]
    print(col)
    print('There are {} tracts identified as outliers.'.format(outliers.shape[0]))

    df_diff[col] = pd.Series(winsorize(df_diff[col], limits = (outliers.shape[0]/df_diff[col].shape[0])))

TotalPop
There are 66 tracts identified as outliers.
Men
There are 160 tracts identified as outliers.
Hispanic
There are 0 tracts identified as outliers.
White
There are 0 tracts identified as outliers.
Black
There are 2367 tracts identified as outliers.
Native
There are 3223 tracts identified as outliers.
Asian
There are 2205 tracts identified as outliers.
Pacific
There are 8847 tracts identified as outliers.
Citizen
There are 2 tracts identified as outliers.
Income
There are 22 tracts identified as outliers.
IncomeErr
There are 223 tracts identified as outliers.
IncomePerCap
There are 221 tracts identified as outliers.
IncomePerCapErr
There are 688 tracts identified as outliers.
Poverty
There are 0 tracts identified as outliers.
ChildPoverty
There are 0 tracts identified as outliers.
Professional
There are 0 tracts identified as outliers.
Service
There are 1 tracts identified as outliers.
Office
There are 1 tracts identified as outliers.
Construction
There are 35 tracts identified as

In [8]:
#Drop null values

df2015.dropna(inplace=True)
df_diff.dropna(inplace=True)

In [9]:
#Create y-variable as categorical poverty change, X variable as all 2015 statistics
#Perform train test split; also create "small" dataset for faster computation during testing
#Create scaled dataset for use when necessary

y = df2015['Poverty_Change']

X = df2015.drop(['State', 'County', 'Poverty_Change', 'Poverty_Change_val', 'Poverty_Change_2c'], axis=1)

# df_small = df2015.sample(frac=0.1)
# y_small = df_small['Poverty_Change_2c']
# X_small = df_small.drop(['State', 'County', 'Poverty_Change', 'Poverty_Change_val', 'Poverty_Change_2c'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
# X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_small,y_small,test_size=0.2)

scaler = StandardScaler()
X_train_scale = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test_scale = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)
# X_train_s_scale = pd.DataFrame(scaler.fit_transform(X_train_s), columns = X_train_s.columns)
# X_test_s_scale = pd.DataFrame(scaler.fit_transform(X_test_s), columns = X_test_s.columns)

In [ ]:
sm = SMOTE()

X_train_res, y_train_res = sm.fit_resample(X_train_scale, y_train)

In [ ]:
#Perform dummy classification for baseline

dummy = DummyClassifier(strategy = 'most_frequent')
dummy.fit(X_train, y_train)

print(dummy.score(X_train, y_train))
print(dummy.score(X_test, y_test))

plot_confusion_matrix(dummy, X_test, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit = LogisticRegression(max_iter=10000)

logit.fit(X_train_scale, y_train)
print(logit.score(X_train_scale, y_train))
print(logit.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit, X_test_scale, y_test, cmap='Blues')

In [ ]:
# y_pred = logit.predict_proba(X_test)
# y_pred[:,1]

In [ ]:
# y_pred_cat = np.where(y_pred[:,1] >= 0.5, 1,0)
# y_pred_cat

In [ ]:
start_time = time.time()

logit_smote = LogisticRegression(max_iter=10000)

logit_smote.fit(X_train_res, y_train_res)
print(logit_smote.score(X_train_res, y_train_res))
print(logit_smote.score(X_train_scale, y_train))
print(logit_smote.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga = LogisticRegression(max_iter=10000, solver='saga')

logit_smote_saga.fit(X_train_res, y_train_res)
print(logit_smote_saga.score(X_train_res, y_train_res))
print(logit_smote_saga.score(X_train_scale, y_train))
print(logit_smote_saga.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga2 = LogisticRegression(max_iter=10000, solver='saga', penalty='l1')

logit_smote_saga2.fit(X_train_res, y_train_res)
print(logit_smote_saga2.score(X_train_res, y_train_res))
print(logit_smote_saga2.score(X_train_scale, y_train))
print(logit_smote_saga2.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga2, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga3 = LogisticRegression(max_iter=10000, solver='saga', penalty='elasticnet', l1_ratio=0.5)

logit_smote_saga3.fit(X_train_res, y_train_res)
print(logit_smote_saga3.score(X_train_res, y_train_res))
print(logit_smote_saga3.score(X_train_scale, y_train))
print(logit_smote_saga3.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga3, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga_ovr = LogisticRegression(max_iter=10000, solver='saga', multi_class='ovr')

logit_smote_saga_ovr.fit(X_train_res, y_train_res)
print(logit_smote_saga_ovr.score(X_train_res, y_train_res))
print(logit_smote_saga_ovr.score(X_train_scale, y_train))
print(logit_smote_saga_ovr.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga_ovr, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga_ovr2 = LogisticRegression(max_iter=10000, solver='saga', multi_class='ovr', penalty='l1')

logit_smote_saga_ovr2.fit(X_train_res, y_train_res)
print(logit_smote_saga_ovr2.score(X_train_res, y_train_res))
print(logit_smote_saga_ovr2.score(X_train_scale, y_train))
print(logit_smote_saga_ovr2.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga_ovr2, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

logit_smote_saga_ovr3 = LogisticRegression(max_iter=10000, solver='saga', multi_class='ovr', penalty='elasticnet', l1_ratio=0.5)

logit_smote_saga_ovr3.fit(X_train_res, y_train_res)
print(logit_smote_saga_ovr3.score(X_train_res, y_train_res))
print(logit_smote_saga_ovr3.score(X_train_scale, y_train))
print(logit_smote_saga_ovr3.score(X_test_scale, y_test))

print("--- %s seconds ---" % (time.time() - start_time))

plot_confusion_matrix(logit_smote_saga_ovr3, X_test_scale, y_test, cmap='Blues')

In [ ]:
parameters = {'C':stats.uniform(0.1, 100)}
parameters_elasticnet = {'C':stats.uniform(0.1, 0.9), 'l1_ratio':stats.uniform(0.1, 0.9)}

In [ ]:
start_time = time.time()

grid_smote_saga = RandomizedSearchCV(logit_smote_saga, parameters, cv=3, n_iter=30, n_jobs=2)
grid_smote_saga.fit(X_train_res, y_train_res)

print("--- %s seconds ---" % (time.time() - start_time))

print(grid_smote_saga.best_score_)
print(grid_smote_saga.best_params_)
plot_confusion_matrix(grid_smote_saga.best_estimator_, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

grid_smote_saga2 = RandomizedSearchCV(logit_smote_saga2, parameters, cv=3, n_iter=30, n_jobs=2)
grid_smote_saga2.fit(X_train_res, y_train_res)

print("--- %s seconds ---" % (time.time() - start_time))

print(grid_smote_saga2.best_score_)
print(grid_smote_saga2.best_params_)
plot_confusion_matrix(grid_smote_saga2.best_estimator_, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

grid_smote_saga_ovr = RandomizedSearchCV(logit_smote_saga_ovr, parameters, cv=3, n_iter=30, n_jobs=2)
grid_smote_saga_ovr.fit(X_train_res, y_train_res)

print("--- %s seconds ---" % (time.time() - start_time))

print(grid_smote_saga_ovr.best_score_)
print(grid_smote_saga_ovr.best_params_)
plot_confusion_matrix(grid_smote_saga_ovr.best_estimator_, X_test_scale, y_test, cmap='Blues')

In [ ]:
start_time = time.time()

grid_smote_saga_ovr2 = RandomizedSearchCV(logit_smote_saga_ovr2, parameters, cv=3, n_iter=30, n_jobs=2)
grid_smote_saga_ovr2.fit(X_train_res, y_train_res)

print("--- %s seconds ---" % (time.time() - start_time))

print(grid_smote_saga_ovr2.best_score_)
print(grid_smote_saga_ovr2.best_params_)
plot_confusion_matrix(grid_smote_saga_ovr2.best_estimator_, X_test_scale, y_test, cmap='Blues')

In [ ]:
# start_time = time.time()

# boost_c = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000, max_depth=2, subsample=0.5, max_features='sqrt')

# boost_c.fit(X_train, y_train)

# print("--- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()

# print(boost_c.score(X_train, y_train))
# print(boost_c.score(X_test, y_test))

# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# plot_confusion_matrix(boost_c, X_test, y_test, cmap='Blues')

In [ ]:
# start_time = time.time()

# boost_c_smote = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000, max_depth=3, max_features='sqrt')

# boost_c_smote.fit(X_train_res, y_train_res)

# print("--- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()

# print(boost_c_smote.score(X_train, y_train))
# print(boost_c_smote.score(X_test, y_test))

# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# plot_confusion_matrix(boost_c_smote, X_test, y_test, cmap='Blues')